In [1]:
import pandas as pd

df = pd.read_csv("/Users/robinfeldmann/TopicAnalysisRUWTweets/Topics_10/topics_10.csv")
df = df[["tweetid", "lang", "topics_0"]]

In [3]:
df_sentiments = pd.read_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/Sentiments/all_sentiments.csv"
)
df_sentiments = df_sentiments[["tweetid", "tweetcreatedts", "sentiment_label", "lang"]]

In [10]:
df_merged = df.merge(df_sentiments, left_on="tweetid", right_on="tweetid")

In [ ]:
date = df.iloc[0][OrigDataSchema.TIMESTAMP][:10]

df["weeks"] = (
    df[MetaDataSchema.DATE]
    .apply(pd.to_datetime)
    .apply(
        lambda x: (
            str(x.year) + "-" + str(x.week)
            if len(str(x.week)) == 2
            else str(x.year) + "-0" + str(x.week)
        )
    )
)

In [15]:
a = df_merged.sample(10)
a["tweetcreatedts"].str.slice(0, 10)

10653999    2022-03-18
1841571     2022-12-31
9109416     2022-06-09
8384016     2022-04-21
16854246    2022-05-25
11581420    2022-03-26
5738011     2022-06-02
13645566    2022-07-30
13654974    2023-05-23
1566858     2022-09-17
Name: tweetcreatedts, dtype: object

In [17]:
from tqdm.auto import tqdm

tqdm.pandas()

df_merged = df_merged.assign(date=df_merged["tweetcreatedts"].str.slice(0, 10))

In [20]:
df_merged = df_merged[df_merged["date"].str.len() == 10]

In [26]:
df_merged = df_merged[
    (
        df_merged["date"].str.startswith("2022")
        | df_merged["date"].str.startswith("2023")
    )
]

In [27]:
df_merged = df_merged.assign(
    week=df_merged["date"]
    .progress_apply(pd.to_datetime)
    .apply(
        lambda x: (
            str(x.year) + "-" + str(x.week)
            if len(str(x.week)) == 2
            else str(x.year) + "-0" + str(x.week)
        )
    )
)

100%|██████████| 18289082/18289082 [26:59<00:00, 11293.60it/s]


In [29]:
df_merged.to_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Topics_10/week_sentiment_topic.csv"
)

In [1]:
import pandas as pd

df = pd.read_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Topics_10/week_sentiment_topic.csv"
)

In [3]:
df.head()

,Unnamed: 0,tweetid,lang_x,topics_0,tweetcreatedts,sentiment_label,lang_y,date,week
0,0,1.526352e+18,de,4,2022-05-17 00:00:34.000000,neutral,de,2022-05-17,2022-20
1,1,1.526352e+18,de,8,2022-05-17 00:01:11.000000,negative,de,2022-05-17,2022-20
2,2,1.526352e+18,de,6,2022-05-17 00:02:13.000000,neutral,de,2022-05-17,2022-20
3,3,1.526352e+18,de,3,2022-05-17 00:02:18.000000,negative,de,2022-05-17,2022-20
4,4,1.526353e+18,de,9,2022-05-17 00:03:32.000000,neutral,de,2022-05-17,2022-20


Aggregate Counts for TopicxLanguagexWeek and TopicxSentimentxWeek

In [17]:
df_lang = (
    df.groupby(["week", "topics_0", "lang_x"])
    .count()
    .reset_index()[["week", "topics_0", "lang_x", "tweetid"]]
    .rename(columns={"topics_0": "topic", "lang_x": "lang", "tweetid": "lang_count"})
)

In [21]:
df_lang.to_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Topics_10/Topics_0/lang_agg.csv"
)

In [19]:
df_sentiment = (
    df.groupby(["week", "topics_0", "sentiment_label"])
    .count()
    .reset_index()[["week", "topics_0", "tweetid", "sentiment_label"]]
    .rename(columns={"tweetid": "sentiment_count", "topics_0": "topic"})
)

In [20]:
df_sentiment.to_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Topics_10/Topics_0/sentiments_agg.csv"
)

In [23]:
df.head()

,Unnamed: 0,tweetid,lang_x,topics_0,tweetcreatedts,sentiment_label,lang_y,date,week
0,0,1.526352e+18,de,4,2022-05-17 00:00:34.000000,neutral,de,2022-05-17,2022-20
1,1,1.526352e+18,de,8,2022-05-17 00:01:11.000000,negative,de,2022-05-17,2022-20
2,2,1.526352e+18,de,6,2022-05-17 00:02:13.000000,neutral,de,2022-05-17,2022-20
3,3,1.526352e+18,de,3,2022-05-17 00:02:18.000000,negative,de,2022-05-17,2022-20
4,4,1.526353e+18,de,9,2022-05-17 00:03:32.000000,neutral,de,2022-05-17,2022-20


In [26]:
df.groupby(["week", "topics_0"]).count().reset_index()[
    ["week", "topics_0", "tweetid"]
].rename(columns={"topics_0": "topic", "tweetid": "count"}).to_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Topics_10/Topics_0/count_agg.csv"
)